In [47]:
import snscrape
import pandas as pd
import networkx as nx
import re


In [48]:
df = pd.read_json('elizabeth.json', lines=True)
df.columns

Index(['_type', 'url', 'date', 'content', 'renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel',
       'outlinks', 'tcooutlinks', 'media', 'retweetedTweet', 'quotedTweet',
       'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 'coordinates',
       'place', 'hashtags', 'cashtags'],
      dtype='object')

In [49]:
scrap = df.drop(['_type', 'url', 'date', 'renderedContent', 'id',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel',
       'outlinks', 'tcooutlinks', 'media', 'retweetedTweet', 'quotedTweet',
       'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 'coordinates',
       'place', 'hashtags', 'cashtags'], axis=1)
scrap


,content,user
0,La présidente éthiopienne Sahle-Work Zewde a r...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
1,Queen Elizabeth Portrait Tribute T-Shirt\n #El...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
2,"Answer the question below. Enhance, refine, an...","{'_type': 'snscrape.modules.twitter.User', 'us..."
3,Elizabeth Bennet Literary Candle 👋\n #Candle #...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
4,"Visiblement, le petit #Trudeau a perdu sa gran...","{'_type': 'snscrape.modules.twitter.User', 'us..."
...,...,...
995,"Queen Elizabeth Ornament Mug,Elizabeth Mug,Que...","{'_type': 'snscrape.modules.twitter.User', 'us..."
996,A moment from today. Queen Elizabeth II funera...,"{'_type': 'snscrape.modules.twitter.User', 'us..."
997,"#Diana 's shadow on #Elizabeth 's #funeral, a ...","{'_type': 'snscrape.modules.twitter.User', 'us..."
998,Queen Elizabeth II Is Under Medical Supervisio...,"{'_type': 'snscrape.modules.twitter.User', 'us..."


In [50]:
listuser = []
for row in scrap['user']:
    listuser.append('@'+row['username'])
scrap['user'] = listuser
scrap

,content,user
0,La présidente éthiopienne Sahle-Work Zewde a r...,@rfemassala
1,Queen Elizabeth Portrait Tribute T-Shirt\n #El...,@accidentambula1
2,"Answer the question below. Enhance, refine, an...",@UPSCPractice
3,Elizabeth Bennet Literary Candle 👋\n #Candle #...,@medea_ebooks
4,"Visiblement, le petit #Trudeau a perdu sa gran...",@sergiodde
...,...,...
995,"Queen Elizabeth Ornament Mug,Elizabeth Mug,Que...",@LjburgLzou8ldGO
996,A moment from today. Queen Elizabeth II funera...,@DMann2017
997,"#Diana 's shadow on #Elizabeth 's #funeral, a ...",@AIdreessy
998,Queen Elizabeth II Is Under Medical Supervisio...,@GraceSmartsBlog


In [51]:
text = scrap['content'][1]
text

'Queen Elizabeth Portrait Tribute T-Shirt\n #Elizabeth #Tribute #Portrait #Tshirt #TheTRNDshop\n\n👉https://t.co/q9dHxwIznN https://t.co/oGhs2ApzqC'

In [52]:
mentionnet = nx.DiGraph()

for row in scrap.iterrows():
    author = row[1]['user']
    text = row[1]['content']

    mentions = set(re.findall(r'@(\w+)', text))

    if len(mentions) > 0:
        for u in mentions:
            u = f'@{u}'
            mentionnet.add_edge(author, u)
    

In [53]:
degreecentral = nx.degree_centrality(mentionnet)

In [54]:
print(degreecentral)

{'@Queen_Q1978': 0.003257328990228013, '@tequieremos': 0.003257328990228013, '@Georges37704025': 0.003257328990228013, '@wdeguerpel': 0.003257328990228013, '@gezegendunya75': 0.003257328990228013, '@ProfSFindik': 0.003257328990228013, '@EVERALDATLARGE': 0.006514657980456026, '@AlboMP': 0.003257328990228013, '@SpectatorOz': 0.003257328990228013, '@MaraPer46623254': 0.003257328990228013, '@NancyEs78964067': 0.003257328990228013, '@shikari': 0.02280130293159609, '@RoyalFamily': 0.02280130293159609, '@BBCBreaking': 0.003257328990228013, '@lesbeaann53': 0.003257328990228013, '@FabVeronika': 0.003257328990228013, '@AlexMayer283': 0.003257328990228013, '@JPJarod': 0.003257328990228013, '@KaishaWaldron': 0.003257328990228013, '@Cliff_Reeve': 0.003257328990228013, '@SamuelAsfaw13': 0.003257328990228013, '@afizejem': 0.003257328990228013, '@Pulga_Phil_M': 0.003257328990228013, '@rarible': 0.006514657980456026, '@PbacClub': 0.003257328990228013, '@lifeasafrica': 0.003257328990228013, '@officialAB

In [55]:
dfexport = nx.to_pandas_edgelist(mentionnet)
dfexport.to_csv('queen.csv')